In [1]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from typing import List, Dict, Any
import time
import os
import pickle

In [2]:
class SemanticSearch:
    # def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
    # def __init__(self, model_name: str = "all-MiniLM-L12-v2"):
    # def __init__(self, model_name: str = 'sentence-transformers/all-mpnet-base-v2'):
    # def __init__(self, model_name: str ="BAAI/bge-base-en-v1.5"):
    # def __init__(self, model_name: str = 'intfloat/e5-large-v2'): best
    # def __init__(self, model_name: str = 'sentence-transformers/all-mpnet-base-v2'):
    # def __init__(self, model_name: str = 'thenlper/gte-base'):
    def __init__(self, model_name: str = 'hkunlp/instructor-xl'):

        """Initialize the SemanticSearch class with a pre-trained model"""
        print(f"Loading model: {model_name}")
        self.model = SentenceTransformer(model_name)
        self.database_items = []
        self.database_embeddings = None
        self.embedding_cache_path = "embedding_cache.pkl"
        
    def add_documents(self, documents: List[str], use_cache: bool = True) -> None:
        """Add documents to the search database and encode them"""
        self.database_items = documents
        
        if use_cache and os.path.exists(self.embedding_cache_path):
            try:
                with open(self.embedding_cache_path, 'rb') as f:
                    cached_data = pickle.load(f)
                    if cached_data.get('documents') == documents:
                        print("Using cached embeddings")
                        self.database_embeddings = cached_data.get('embeddings')
                        return
            except Exception as e:
                print(f"Error loading cache: {e}")
        
        # Encode documents to embeddings
        print(f"Encoding {len(documents)} documents...")
        start_time = time.time()
        self.database_embeddings = self.model.encode(documents, show_progress_bar=True)
        print(f"Encoding completed in {time.time() - start_time:.2f} seconds")
        
        # Save embeddings to cache
        if use_cache:
            with open(self.embedding_cache_path, 'wb') as f:
                pickle.dump({
                    'documents': documents,
                    'embeddings': self.database_embeddings
                }, f)
            print("Embeddings cached for future use")
    
    def search(self, query: str, top_n: int = 5) -> List[Dict[str, Any]]:
        """Perform semantic search for a query against the document database"""
        if self.database_embeddings is None or len(self.database_items) == 0:
            raise ValueError("No documents have been added to the search database")
        
        # Encode the query
        query_embedding = self.model.encode(query)
        
        # Calculate cosine similarities
        similarities = util.pytorch_cos_sim(
            query_embedding.reshape(1, -1), 
            self.database_embeddings
        )[0]
        
        top_indices = np.argsort(similarities.numpy())[::-1][:top_n]
        
        results = []
        for idx in top_indices:
            results.append({
                "document": self.database_items[idx],
                "score": float(similarities[idx]),
                "position": len(results) + 1
            })
        
        return results
    
    def batch_search(self, queries: List[str], top_n: int = 5) -> Dict[str, List[Dict[str, Any]]]:
        """Perform semantic search for multiple queries at once"""
        results = {}
        for query in queries:
            results[query] = self.search(query, top_n)
        return results

In [3]:
database_items = [
    "Hospital A in London UK",
    "Medical Center B in London UK", 
    "Clinic C in London UK",
    "London General Hospital",
    "Emergency Care Unit in South London",
    "Royal London Hospital",
    "London Bridge Hospital",
    "Private Healthcare Facility in Central London",
    "St Thomas' Hospital London",
    "Guy's Hospital London UK",
    "University College Hospital London",
    "Healthcare Center in North London",
    "Pediatric Hospital in London",
    "Specialist Cancer Center London UK",
    "Mental Health Facility London"
]

In [4]:
search_engine = SemanticSearch()
search_engine.add_documents(["pediatric medical care london"])

Loading model: hkunlp/instructor-xl


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

Encoding 1 documents...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Encoding completed in 2.10 seconds
Embeddings cached for future use


In [5]:
results = search_engine.search("hospitals in london uk", top_n=5)
print(results)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[{'document': 'pediatric medical care london', 'score': 0.8226864337921143, 'position': 1}]


In [6]:
print(results)

[{'document': 'pediatric medical care london', 'score': 0.8226864337921143, 'position': 1}]


In [7]:
def main():

    
    # Sample queries
    sample_queries = [
        "hospitals in london uk",
        "emergency care in london",
        "pediatric medical care london",
        "private healthcare london",
        "mental health services"
    ]
    
    search_engine = SemanticSearch()
    search_engine.add_documents(database_items)
    
    # Perform individual search
    print("\n--- Individual Search ---")
    results = search_engine.search("hospitals in london uk", top_n=1)
    
    print(f"Query: hospitals in london uk")
    for result in results:
        print(f"- {result['document']} (Score: {result['score']:.4f})")
    
    # Perform batch search
    print("\n--- Batch Search ---")
    batch_results = search_engine.batch_search(sample_queries, top_n=3)
    
    for query, results in batch_results.items():
        print(f"\nQuery: {query}")
        for result in results:
            print(f"- {result['document']} (Score: {result['score']:.4f})")

if __name__ == "__main__":
    main()

Loading model: hkunlp/instructor-xl
Encoding 15 documents...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Encoding completed in 0.17 seconds
Embeddings cached for future use

--- Individual Search ---


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: hospitals in london uk
- Hospital A in London UK (Score: 0.9039)

--- Batch Search ---


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: hospitals in london uk
- Hospital A in London UK (Score: 0.9039)
- Pediatric Hospital in London (Score: 0.8796)
- Royal London Hospital (Score: 0.8769)

Query: emergency care in london
- Emergency Care Unit in South London (Score: 0.9090)
- Hospital A in London UK (Score: 0.8380)
- Healthcare Center in North London (Score: 0.8133)

Query: pediatric medical care london
- Pediatric Hospital in London (Score: 0.9321)
- Healthcare Center in North London (Score: 0.8202)
- Medical Center B in London UK (Score: 0.8122)

Query: private healthcare london
- Private Healthcare Facility in Central London (Score: 0.9359)
- Healthcare Center in North London (Score: 0.8246)
- Hospital A in London UK (Score: 0.8106)

Query: mental health services
- Mental Health Facility London (Score: 0.7380)
- Healthcare Center in North London (Score: 0.6336)
- Private Healthcare Facility in Central London (Score: 0.6125)


In [3]:
# import torch
# from sentence_transformers import SentenceTransformer
# from typing import List

# def process_model(model_name: str, queries: List[str], documents: List[str], top_n: int = 5):
#     """
#     Load a model, perform semantic search, and release GPU memory.
#     """
#     print(f"Processing with model: {model_name}")
    
#     # Load the model
#     model = SentenceTransformer(model_name)
    
#     # Encode documents
#     print("Encoding documents...")
#     document_embeddings = model.encode(documents, show_progress_bar=True)
    
#     # Process each query
#     results = []
#     for query in queries:
#         print(f"Processing query: {query}")
#         query_embedding = model.encode(query)
        
#         # Calculate cosine similarities
#         similarities = torch.nn.functional.cosine_similarity(
#             torch.tensor(query_embedding).unsqueeze(0).cuda(),
#             torch.tensor(document_embeddings).cuda()
#         )
        
#         # Get top N results
#         top_indices = torch.argsort(similarities, descending=True)[:top_n]
#         query_results = [
#             {"document": documents[idx], "score": float(similarities[idx])}
#             for idx in top_indices
#         ]
#         results.append({"query": query, "results": query_results})
    
#     # Clear GPU memory
#     del model
#     torch.cuda.empty_cache()
    
#     return results

# def main():
#     # Define models, queries, and documents
#     models = [
#         "sentence-transformers/all-MiniLM-L6-v2",
#         "sentence-transformers/all-mpnet-base-v2",
#         "hkunlp/instructor-xl"
#     ]
    
#     queries = [
#         "hospitals in london uk",
#         "emergency care in london",
#         "pediatric medical care london",
#         "private healthcare london",
#         "mental health services"
#     ]
    
#     documents = ["pediatric medical care london"]
    
#     # Process each model sequentially
#     for model_name in models:
#         results = process_model(model_name, queries, documents)
        
#         # Print results for each model
#         print(f"\nResults for model: {model_name}")
#         for result in results:
#             print(f"Query: {result['query']}")
#             for res in result['results']:
#                 print(f"- {res['document']} (Score: {res['score']:.4f})")

# if __name__ == "__main__":
#     main()


import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from typing import List, Dict, Any, Tuple
import time
import os
import pickle
import gc
import torch

class SemanticSearch:
    def __init__(self, model_name: str):
        """Initialize the SemanticSearch class with a pre-trained model"""
        print(f"Loading model: {model_name}")
        self.model_name = model_name
        self.model = SentenceTransformer(model_name)
        self.database_items = []
        self.database_embeddings = None
        self.embedding_cache_path = f"embedding_cache_{model_name.replace('/', '_')}.pkl"
        
    def add_documents(self, documents: List[str], use_cache: bool = True) -> float:
        """Add documents to the search database and encode them, returns encoding time"""
        self.database_items = documents
        
        if use_cache and os.path.exists(self.embedding_cache_path):
            try:
                with open(self.embedding_cache_path, 'rb') as f:
                    cached_data = pickle.load(f)
                    if cached_data.get('documents') == documents:
                        print("Using cached embeddings")
                        self.database_embeddings = cached_data.get('embeddings')
                        return 0.0  # No encoding time when using cache
            except Exception as e:
                print(f"Error loading cache: {e}")
        
        # Encode documents to embeddings
        print(f"Encoding {len(documents)} documents...")
        start_time = time.time()
        self.database_embeddings = self.model.encode(documents, show_progress_bar=True)
        encoding_time = time.time() - start_time
        print(f"Encoding completed in {encoding_time:.2f} seconds")
        
        # Save embeddings to cache
        if use_cache:
            with open(self.embedding_cache_path, 'wb') as f:
                pickle.dump({
                    'documents': documents,
                    'embeddings': self.database_embeddings
                }, f)
            print("Embeddings cached for future use")
            
        return encoding_time
    
    def search(self, query: str, top_n: int = 5) -> List[Dict[str, Any]]:
        """Perform semantic search for a query against the document database"""
        if self.database_embeddings is None or len(self.database_items) == 0:
            raise ValueError("No documents have been added to the search database")
        
        # Encode the query
        query_embedding = self.model.encode(query)
        
        # Calculate cosine similarities
        similarities = util.pytorch_cos_sim(
            query_embedding.reshape(1, -1), 
            self.database_embeddings
        )[0]
        
        top_indices = np.argsort(similarities.numpy())[::-1][:top_n]
        
        results = []
        for idx in top_indices:
            results.append({
                "document": self.database_items[idx],
                "score": float(similarities[idx]),
                "position": len(results) + 1
            })
        
        return results
    
    def batch_search(self, queries: List[str], top_n: int = 5) -> Tuple[Dict[str, List[Dict[str, Any]]], float]:
        """Perform semantic search for multiple queries at once, returns results and search time"""
        results = {}
        start_time = time.time()
        for query in queries:
            results[query] = self.search(query, top_n)
        search_time = time.time() - start_time
        return results, search_time
    
    def cleanup(self):
        """Clean up model to free memory"""
        del self.model
        if hasattr(self, 'database_embeddings'):
            del self.database_embeddings
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
        print(f"Model {self.model_name} has been cleaned up")


def evaluate_model(model_name, database_items, sample_queries, results_file_path):
    """Evaluate a single model and save results to file"""
    print(f"\n{'='*50}")
    print(f"Evaluating model: {model_name}")
    print(f"{'='*50}")
    
    try:
        # Initialize search engine and measure time
        start_time = time.time()
        search_engine = SemanticSearch(model_name)
        initialization_time = time.time() - start_time
        
        # Add documents and get encoding time
        encoding_time = search_engine.add_documents(database_items)
        
        # Perform batch search and get search time
        batch_results, search_time = search_engine.batch_search(sample_queries, top_n=3)
        
        # Calculate average score for each query as a performance metric
        performance_metrics = {}
        for query, results in batch_results.items():
            if results:
                avg_score = sum(r['score'] for r in results) / len(results)
                top_score = results[0]['score'] if results else 0
                performance_metrics[query] = {
                    'avg_score': avg_score,
                    'top_score': top_score
                }
        
        avg_top_score = sum(metrics['top_score'] for metrics in performance_metrics.values()) / len(performance_metrics)
        
        # Save results to file
        with open(results_file_path, 'a') as f:
            f.write(f"\n{'='*50}\n")
            f.write(f"Model: {model_name}\n")
            f.write(f"{'='*50}\n")
            
            # Timing metrics
            f.write("TIMING METRICS:\n")
            f.write(f"Initialization time: {initialization_time:.2f} seconds\n")
            f.write(f"Encoding time: {encoding_time:.2f} seconds\n")
            f.write(f"Search time for {len(sample_queries)} queries: {search_time:.2f} seconds\n")
            f.write(f"Average search time per query: {search_time/len(sample_queries):.4f} seconds\n\n")
            
            # Performance metrics
            f.write("PERFORMANCE METRICS:\n")
            f.write(f"Average top result score: {avg_top_score:.4f}\n\n")
            
            # Detailed search results
            f.write("SEARCH RESULTS:\n")
            for query, results in batch_results.items():
                f.write(f"\nQuery: {query}\n")
                f.write(f"  Average score: {performance_metrics[query]['avg_score']:.4f}\n")
                f.write(f"  Top score: {performance_metrics[query]['top_score']:.4f}\n")
                for result in results:
                    f.write(f"  - {result['document']} (Score: {result['score']:.4f})\n")
            
            f.write(f"\n{'-'*50}\n")
        
        # Print summary
        print(f"Model: {model_name}")
        print(f"Initialization time: {initialization_time:.2f} seconds")
        print(f"Encoding time: {encoding_time:.2f} seconds")
        print(f"Search time: {search_time:.2f} seconds")
        print(f"Average top score: {avg_top_score:.4f}")
        
        # Clean up model
        search_engine.cleanup()
        return True
        
    except Exception as e:
        with open(results_file_path, 'a') as f:
            f.write(f"\n{'='*50}\n")
            f.write(f"Model: {model_name}\n")
            f.write(f"ERROR: {str(e)}\n")
            f.write(f"\n{'-'*50}\n")
        print(f"Error evaluating model {model_name}: {e}")
        return False


def main():
    # Database items
    database_items = [
        "Hospital A in London UK",
        "Medical Center B in London UK", 
        "Clinic C in London UK",
        "London General Hospital",
        "Emergency Care Unit in South London",
        "Royal London Hospital",
        "London Bridge Hospital",
        "Private Healthcare Facility in Central London",
        "St Thomas' Hospital London",
        "Guy's Hospital London UK",
        "University College Hospital London",
        "Healthcare Center in North London",
        "Pediatric Hospital in London",
        "Specialist Cancer Center London UK",
        "Mental Health Facility London"
    ]
    
    # Sample queries
    sample_queries = [
        "hospitals in london uk",
        "emergency care in london",
        "pediatric medical care london",
        "private healthcare london",
        "mental health services"
    ]
    
    # List of models to evaluate
    models = [
        "all-MiniLM-L6-v2",
        "all-MiniLM-L12-v2",
        "sentence-transformers/all-mpnet-base-v2",
        "BAAI/bge-base-en-v1.5",
        "intfloat/e5-large-v2",
        "thenlper/gte-base",
        "hkunlp/instructor-xl"
    ]
    
    # Create results directory if it doesn't exist
    os.makedirs("/kaggle/working", exist_ok=True)
    
    # Create results file
    results_file_path = "/kaggle/working/model_evaluation_results.txt"
    with open(results_file_path, 'w') as f:
        f.write("SEMANTIC SEARCH MODEL EVALUATION RESULTS\n")
        f.write(f"Date: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Number of documents: {len(database_items)}\n")
        f.write(f"Number of queries: {len(sample_queries)}\n\n")
        
        f.write("DATABASE ITEMS:\n")
        for i, item in enumerate(database_items):
            f.write(f"{i+1}. {item}\n")
        f.write("\n")
        
        f.write("SAMPLE QUERIES:\n")
        for i, query in enumerate(sample_queries):
            f.write(f"{i+1}. {query}\n")
        f.write("\n")
    
    # Evaluate each model
    successful_models = 0
    for model_name in models:
        if evaluate_model(model_name, database_items, sample_queries, results_file_path):
            successful_models += 1
            
    # Write summary
    with open(results_file_path, 'a') as f:
        f.write(f"\n{'='*50}\n")
        f.write(f"SUMMARY\n")
        f.write(f"{'='*50}\n")
        f.write(f"Total models evaluated: {len(models)}\n")
        f.write(f"Successful evaluations: {successful_models}\n")
        f.write(f"Failed evaluations: {len(models) - successful_models}\n")
        
    print(f"\nEvaluation complete. Results saved to {results_file_path}")
    print(f"Successful models: {successful_models}/{len(models)}")


if __name__ == "__main__":
    main()



Evaluating model: all-MiniLM-L6-v2
Loading model: all-MiniLM-L6-v2
Encoding 15 documents...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Encoding completed in 0.06 seconds
Embeddings cached for future use


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Model: hkunlp/instructor-xl
Initialization time: 4.16 seconds
Encoding time: 0.16 seconds
Search time: 0.28 seconds
Average top score: 0.8838
Model hkunlp/instructor-xl has been cleaned up

Evaluation complete. Results saved to /kaggle/working/model_evaluation_results.txt
Successful models: 7/7
